In [68]:
# !pip install scikit-learn
# !pip install tensorflow
# !pip install matplotlib
# !pip install pandas
# !pip install opencv-python

In [69]:
# import h5py
import datetime
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.utils import shuffle
from sklearn.utils.class_weight import compute_class_weight
%matplotlib inline

In [70]:
import os
import cv2
import h5py
import numpy as np

def load_data(path_prefix, dataset_name, splits=['train', 'val', 'test']):
    X, y = {}, {}

    IMG_SIZE = 224 if 'RAFDB' in dataset_name else 120
    splits = ['train', 'test'] if 'RAFDB' in dataset_name else splits

    Label_Mapping = {0:'Angry', 1:'Disgust', 2:'Fear', 3:'Happy', 4:'Sad', 5:'Surprise', 6:'Neutral', 7:'Contempt'}
    
    for split in splits:
        PATH = os.path.join(path_prefix, dataset_name, split)
        X[split], y[split] = [], []
        for classes in Label_Mapping.values():
            class_path = os.path.join(PATH, classes)
            class_numeric = next((k for k, v in Label_Mapping.items() if v == classes), None)
            # print(f"{classes}_{class_numeric}_{class_path}")
            # print('---------------')

            try:
                for sample in os.listdir(class_path):
                    sample_path = os.path.join(class_path, sample)
                    image = cv2.imread(sample_path, cv2.IMREAD_COLOR)
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
                    X[split].append(image)
                    y[split].append(class_numeric)
            except:
                print(f"Dataset doesn't have class {classes}, therefore skipping.")
    
    # Convert to numpy arrays
    for split in splits:
        X[split] = np.array(X[split])
        y[split] = np.array(y[split])
        
    return X, y

In [71]:
# X, y = load_data('../../Datasets', dataset_name='JAFFE_Structured', splits=['train', 'validation', 'test'])

# with h5py.File('ferp.h5', 'w') as dataset: 
#     for split in X.keys():
#         dataset.create_dataset(f'X_{split}', data=X[split])
#         dataset.create_dataset(f'y_{split}', data=y[split])

# del X, y

In [72]:
# import pandas as pd
# import numpy as np
# from sklearn.utils import shuffle
# from sklearn.utils.class_weight import compute_class_weight
# from PIL import Image
# import os

# data_path = '../../Datasets/JAFFE_Structured'

# # Define dataset paths
# train_dir = f'{data_path}/train'
# valid_dir = f'{data_path}/validation'
# test_dir = f'{data_path}/test'

# def load_data(directory, img_size=(120, 120)):
#     """
#     Load images and labels from a directory containing a labels.csv file.
#     Args:
#         directory: Path to dataset split directory
#         img_size: Target size for image resizing
#     Returns:
#         Tuple of (images array, labels array)
#     """
#     # Load CSV with image filenames and labels
#     labels_path = os.path.join(directory, 'labels.csv')
#     df = pd.read_csv(labels_path)
    
#     # Adjust these column names if your CSV uses different headers
#     filenames = df['filename']
#     labels = df['label'].values
    
#     images = []
#     for filename in filenames:
#         img_path = os.path.join(directory, filename)
        
#         # Load and preprocess image
#         img = Image.open(img_path).convert('RGB')
#         img = img.resize(img_size)
#         img_array = np.array(img)
        
#         # Optional normalization (uncomment if needed)
#         # img_array = img_array / 255.0
        
#         images.append(img_array)
    
#     return np.array(images), labels

# # Load all datasets
# X_train, y_train = load_data(train_dir)
# X_valid, y_valid = load_data(valid_dir)
# X_test, y_test = load_data(test_dir)

# # Shuffle training data
# X_train, y_train = shuffle(X_train, y_train, random_state=42)

# # Print shapes
# print("Shape of train_sample: {}".format(X_train.shape))
# print("Shape of train_label: {}".format(y_train.shape))
# print("Shape of valid_sample: {}".format(X_valid.shape))
# print("Shape of valid_label: {}".format(y_valid.shape))
# print("Shape of test_sample: {}".format(X_test.shape))
# print("Shape of test_label: {}".format(y_test.shape))

# # Calculate class weights
# class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
# class_weights = dict(enumerate(class_weights))

In [73]:
# X_train, y_train = load_data('../../Datasets', dataset_name='JAFFE_Structured', splits=['train'])
# print(X_train)

In [85]:
NUM_CLASSES = 6
IMG_SHAPE = (120, 120, 3)
BATCH_SIZE = 8

TRAIN_EPOCH = 1 #00
TRAIN_LR = 1e-3
TRAIN_ES_PATIENCE = 0#5
TRAIN_LR_PATIENCE = 3
TRAIN_MIN_LR = 1e-6
TRAIN_DROPOUT = 0.1

FT_EPOCH = 500
FT_LR = 1e-5
FT_LR_DECAY_STEP = 40.0 #80.0
FT_LR_DECAY_RATE = 1
FT_ES_PATIENCE = 20
FT_DROPOUT = 0.2

ES_LR_MIN_DELTA = 0.003

In [75]:
X, y = load_data('../../Datasets', dataset_name='JAFFE_Structured', splits=['train', 'validation', 'test'])

X_train, y_train = shuffle(X['train'], y['train'])
X_valid, y_valid = X['validation'], y['validation']
X_test, y_test = X['test'], y['test']

# X_train, y_train = shuffle(X_train, y_train)

print("Shape of train_sample: {}".format(X_train.shape))
print("Shape of train_label: {}".format(y_train.shape))
print("Shape of valid_sample: {}".format(X_valid.shape))
print("Shape of valid_label: {}".format(y_valid.shape))
print("Shape of test_sample: {}".format(X_test.shape))
print("Shape of test_label: {}".format(y_test.shape))

class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(class_weights))
print(class_weights)

Dataset doesn't have class Neutral, therefore skipping.
Dataset doesn't have class Contempt, therefore skipping.
Dataset doesn't have class Neutral, therefore skipping.
Dataset doesn't have class Contempt, therefore skipping.
Dataset doesn't have class Neutral, therefore skipping.
Dataset doesn't have class Contempt, therefore skipping.
Shape of train_sample: (128, 120, 120, 3)
Shape of train_label: (128,)
Shape of valid_sample: (41, 120, 120, 3)
Shape of valid_label: (41,)
Shape of test_sample: (44, 120, 120, 3)
Shape of test_label: (44,)
{0: np.float64(1.1851851851851851), 1: np.float64(0.8888888888888888), 2: np.float64(4.266666666666667), 3: np.float64(0.6274509803921569), 4: np.float64(0.927536231884058), 5: np.float64(0.8888888888888888)}


In [91]:
# Model Building
input_layer = tf.keras.Input(shape=IMG_SHAPE, name='universal_input')
sample_resizing = tf.keras.layers.Resizing(224, 224, name="resize")
data_augmentation = tf.keras.Sequential([tf.keras.layers.RandomFlip(mode='horizontal'), 
                                        tf.keras.layers.RandomContrast(factor=0.3)], name="augmentation")
preprocess_input = tf.keras.applications.mobilenet.preprocess_input

backbone = tf.keras.applications.mobilenet.MobileNet(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
backbone.trainable = False
base_model = tf.keras.Model(backbone.input, backbone.layers[-29].output, name='base_model')

self_attention = tf.keras.layers.Attention(use_scale=True, name='attention')
patch_extraction = tf.keras.Sequential([
    tf.keras.layers.SeparableConv2D(256, kernel_size=4, strides=4, padding='same', activation='relu'), 
    tf.keras.layers.SeparableConv2D(256, kernel_size=2, strides=2, padding='valid', activation='relu'), 
    tf.keras.layers.Conv2D(256, kernel_size=1, strides=1, padding='valid', activation='relu')
], name='patch_extraction')
global_average_layer = tf.keras.layers.GlobalAveragePooling2D(name='gap')
pre_classification = tf.keras.Sequential([tf.keras.layers.Dense(32, activation='relu'), 
                                          tf.keras.layers.BatchNormalization()], name='pre_classification')
prediction_layer = tf.keras.layers.Dense(NUM_CLASSES, activation="softmax", name='classification_head')

inputs = input_layer
x = sample_resizing(inputs)
x = data_augmentation(x)
x = preprocess_input(x)
x = base_model(x, training=False)
x = patch_extraction(x)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(TRAIN_DROPOUT)(x)
x = pre_classification(x)
x = tf.keras.layers.Reshape((1, 32))(x) #----
x = self_attention([x, x])
x = tf.keras.layers.Reshape((32,))(x)  #----
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs, name='train-head')
model.compile(optimizer=keras.optimizers.Adam(learning_rate=TRAIN_LR, global_clipnorm=3.0), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Training Procedure
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=TRAIN_ES_PATIENCE, min_delta=ES_LR_MIN_DELTA, restore_best_weights=True)
learning_rate_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', patience=TRAIN_LR_PATIENCE, verbose=0, min_delta=ES_LR_MIN_DELTA, min_lr=TRAIN_MIN_LR)

class_weights = {cls: float(weight) for cls, weight in class_weights.items()}

history = model.fit(X_train, y_train, epochs=TRAIN_EPOCH, batch_size=BATCH_SIZE, validation_data=(X_valid, y_valid), verbose=0, 
                    class_weight=class_weights, callbacks=[early_stopping_callback, learning_rate_callback])



# history = model.fit(X_train, y_train, epochs=TRAIN_EPOCH, batch_size=BATCH_SIZE, validation_data=(X_valid, y_valid), verbose=0, callbacks=[early_stopping_callback, learning_rate_callback])
test_loss, test_acc = model.evaluate(X_test, y_test)

# Model Finetuning
print("\nFinetuning ...")
unfreeze = 59
base_model.trainable = True
fine_tune_from = len(base_model.layers) - unfreeze
for layer in base_model.layers[:fine_tune_from]:
    layer.trainable = False
for layer in base_model.layers[fine_tune_from:]:
    if isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = False

inputs = input_layer
x = sample_resizing(inputs)
x = data_augmentation(x)
x = preprocess_input(x)
x = base_model(x, training=False)
x = patch_extraction(x)
x = tf.keras.layers.SpatialDropout2D(FT_DROPOUT)(x)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(FT_DROPOUT)(x)
x = pre_classification(x)
x = tf.keras.layers.Reshape((1, 32))(x)  #----
x = self_attention([x, x])
x = tf.keras.layers.Reshape((32,))(x)  #----
x = tf.keras.layers.Dropout(FT_DROPOUT)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs, name='finetune-backbone')
model.compile(optimizer=keras.optimizers.Adam(learning_rate=FT_LR, global_clipnorm=3.0), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Custom function to ensure the scheduler output is a float
def lr_schedule(epoch, lr):
    return float(scheduler(epoch))

# Training Procedure
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', min_delta=ES_LR_MIN_DELTA, patience=FT_ES_PATIENCE, restore_best_weights=True)
scheduler = keras.optimizers.schedules.InverseTimeDecay(initial_learning_rate=FT_LR, decay_steps=FT_LR_DECAY_STEP, decay_rate=FT_LR_DECAY_RATE)
# scheduler_callback = tf.keras.callbacks.LearningRateScheduler(schedule=scheduler)
scheduler_callback = tf.keras.callbacks.LearningRateScheduler(schedule=lr_schedule)

history_finetune = model.fit(X_train, y_train, epochs=FT_EPOCH, batch_size=BATCH_SIZE, validation_data=(X_valid, y_valid), verbose=0, 
                             initial_epoch=history.epoch[-TRAIN_ES_PATIENCE], callbacks=[early_stopping_callback, scheduler_callback, tensorboard_callback])
test_loss, test_acc = model.evaluate(X_test, y_test)
model.save('model.h5')

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step - accuracy: 0.0256 - loss: 1.7947

Finetuning ...
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step - accuracy: 0.2850 - loss: 1.7662


In [ ]:
# # Model Building
# input_layer = tf.keras.Input(shape=IMG_SHAPE, name='universal_input')
# # sample_resizing = tf.keras.layers.experimental.preprocessing.Resizing(224, 224, name="resize")

# sample_resizing = tf.keras.layers.Resizing(224, 224, name="resize")

# data_augmentation = tf.keras.Sequential([tf.keras.layers.RandomFlip(mode='horizontal'), 
#                                         tf.keras.layers.RandomContrast(factor=0.3)], name="augmentation")
# preprocess_input = tf.keras.applications.mobilenet.preprocess_input

# backbone = tf.keras.applications.mobilenet.MobileNet(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
# backbone.trainable = False
# base_model = tf.keras.Model(backbone.input, backbone.layers[-29].output, name='base_model')

# self_attention = tf.keras.layers.Attention(use_scale=True, name='attention')
# patch_extraction = tf.keras.Sequential([
#     tf.keras.layers.SeparableConv2D(256, kernel_size=4, strides=4, padding='same', activation='relu'), 
#     tf.keras.layers.SeparableConv2D(256, kernel_size=2, strides=2, padding='valid', activation='relu'), 
#     tf.keras.layers.Conv2D(256, kernel_size=1, strides=1, padding='valid', activation='relu')
# ], name='patch_extraction')
# global_average_layer = tf.keras.layers.GlobalAveragePooling2D(name='gap')
# pre_classification = tf.keras.Sequential([tf.keras.layers.Dense(32, activation='relu'), 
#                                           tf.keras.layers.BatchNormalization()], name='pre_classification')
# prediction_layer = tf.keras.layers.Dense(NUM_CLASSES, activation="softmax", name='classification_head')

inputs = input_layer
x = sample_resizing(inputs)
x = data_augmentation(x)
x = preprocess_input(x)
x = base_model(x, training=False)
x = patch_extraction(x)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(TRAIN_DROPOUT)(x)
x = pre_classification(x)
x = tf.keras.layers.Reshape((1, 32))(x) ##---
x = self_attention([x, x])
x = tf.keras.layers.Reshape((32,))(x) #---
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs, name='train-head')
model.compile(optimizer=keras.optimizers.Adam(learning_rate=TRAIN_LR, global_clipnorm=3.0), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# # Training Procedure
# early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=TRAIN_ES_PATIENCE, min_delta=ES_LR_MIN_DELTA, restore_best_weights=True)
# learning_rate_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', patience=TRAIN_LR_PATIENCE, verbose=0, min_delta=ES_LR_MIN_DELTA, min_lr=TRAIN_MIN_LR)
# history = model.fit(X_train, y_train, epochs=TRAIN_EPOCH, batch_size=BATCH_SIZE, validation_data=(X_valid, y_valid), verbose=0, class_weight=class_weights, callbacks=[early_stopping_callback, learning_rate_callback])
# test_loss, test_acc = model.evaluate(X_test, y_test)

# # Model Finetuning
# print("\nFinetuning ...")
# unfreeze = 59
# base_model.trainable = True
# fine_tune_from = len(base_model.layers) - unfreeze
# for layer in base_model.layers[:fine_tune_from]:
#     layer.trainable = False
# for layer in base_model.layers[fine_tune_from:]:
#     if isinstance(layer, tf.keras.layers.BatchNormalization):
#         layer.trainable = False

# inputs = input_layer
# x = sample_resizing(inputs)
# x = data_augmentation(x)
# x = preprocess_input(x)
# x = base_model(x, training=False)
# x = patch_extraction(x)
# x = tf.keras.layers.SpatialDropout2D(FT_DROPOUT)(x)
# x = global_average_layer(x)
# x = tf.keras.layers.Dropout(FT_DROPOUT)(x)
# x = pre_classification(x)
# x = self_attention([x, x])
# x = tf.keras.layers.Dropout(FT_DROPOUT)(x)
# outputs = prediction_layer(x)
# model = tf.keras.Model(inputs, outputs, name='finetune-backbone')
# model.compile(optimizer=keras.optimizers.Adam(learning_rate=FT_LR, global_clipnorm=3.0), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# # Training Procedure
# log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
# early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', min_delta=ES_LR_MIN_DELTA, patience=FT_ES_PATIENCE, restore_best_weights=True)
# scheduler = keras.optimizers.schedules.InverseTimeDecay(initial_learning_rate=FT_LR, decay_steps=FT_LR_DECAY_STEP, decay_rate=FT_LR_DECAY_RATE)
# scheduler_callback = tf.keras.callbacks.LearningRateScheduler(schedule=scheduler)

# history_finetune = model.fit(X_train, y_train, epochs=FT_EPOCH, batch_size=BATCH_SIZE, validation_data=(X_valid, y_valid), verbose=0, 
#                              initial_epoch=history.epoch[-TRAIN_ES_PATIENCE], callbacks=[early_stopping_callback, scheduler_callback, tensorboard_callback])
# test_loss, test_acc = model.evaluate(X_test, y_test)
# model.save('model.h5')

c:\MastersRepos\Deep-Learning-For-Computer-Vision\Models\PAtt-Lite\.venv\lib\site-packages\keras\src\ops\nn.py:908: UserWarning: You are using a softmax over axis -1 of a tensor of shape (8, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
c:\MastersRepos\Deep-Learning-For-Computer-Vision\Models\PAtt-Lite\.venv\lib\site-packages\keras\src\ops\nn.py:908: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - accuracy: 0.1629 - loss: 1.7885

Finetuning ...


IndexError: list index out of range